# Features extraction

In this file, the pre-trained model (https://github.com/csho33/bacteria-ID) is used as a features extractor.

## Librairies

In [1]:
from data_loader import load_data
import os, sys
import random
import numpy as np
import torch
from collections import OrderedDict
from extractor import Extractor

## Pre-trained models
In this part, the three provided pre-trained models are load.

In [2]:
os.chdir(os.getcwd())
models_dir = 'Bacteria_TL'
sys.path.append(models_dir)

models = ['pretrained_model.ckpt', 'finetuned_model.ckpt', 'clinical_pretrained_model.ckpt']

## Dataset
This part is dedicated to the dataset. The data will be load and split into train, validation and test sets.

### Load data

In [3]:
X, Y, groups = load_data()

### Split into train, validation and test sets
The split operation is doing as follow :
   - 60% of the patients data are take for the train set
   - 20% are take for the validation set
   - 20% are take for the test set

In [4]:
selected_groups = random.sample(range(0, 30), 30)
train_groups = selected_groups[:18]
validation_groups = selected_groups[18:24]
test_groups = selected_groups[24:]

In [5]:
def createSplit(X, Y, groups, groups_selected):
    X_split = []
    Y_split = []
    for i in range(len(X)):
        if(groups[i] in groups_selected):
            X_split.append(X[i])
            Y_split.append(Y[i])
    return np.array(X_split), np.array(Y_split)

In [6]:
X_train, Y_train = createSplit(X, Y, groups, train_groups)
X_validation, Y_validation = createSplit(X, Y, groups, validation_groups)
X_test, Y_test = createSplit(X, Y, groups, test_groups)

## Features extractors
This is the part where the different models are modified in order to be used as features extractor.

### Pre-trained models parameters
General parameters of pre-trained models are defined here.

In [7]:
base_layers = 6
hidden_size = 100
block_size = 2
input_dim = 1174
in_channels = 64
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()
base_to_remove = ['linear.weight', 'linear.bias']

The layers belonging to the two last blocks of the original CNN are listed below

In [8]:
BLOCK6 = ['encoder.5.0.conv1.weight', 'encoder.5.0.bn1.weight', 'encoder.5.0.bn1.bias',
          'encoder.5.0.bn1.running_mean', 'encoder.5.0.bn1.running_var', 'encoder.5.0.conv2.weight',
          'encoder.5.0.bn2.weight', 'encoder.5.0.bn2.bias', 'encoder.5.0.bn2.running_mean', 
          'encoder.5.0.bn2.running_var', 'encoder.5.0.shortcut.0.weight', 
          'encoder.5.0.shortcut.1.weight', 'encoder.5.0.shortcut.1.bias', 
          'encoder.5.0.shortcut.1.running_mean', 'encoder.5.0.shortcut.1.running_var', 
          'encoder.5.1.conv1.weight', 'encoder.5.1.bn1.weight', 'encoder.5.1.bn1.bias',
          'encoder.5.1.bn1.running_mean', 'encoder.5.1.bn1.running_var', 'encoder.5.1.conv2.weight',
          'encoder.5.1.bn2.weight', 'encoder.5.1.bn2.bias', 'encoder.5.1.bn2.running_mean',
          'encoder.5.1.bn2.running_var']

BLOCK5 = ['encoder.4.0.conv1.weight', 'encoder.4.0.bn1.weight', 'encoder.4.0.bn1.bias',
          'encoder.4.0.bn1.running_mean', 'encoder.4.0.bn1.running_var', 'encoder.4.0.conv2.weight',
          'encoder.4.0.bn2.weight', 'encoder.4.0.bn2.bias', 'encoder.4.0.bn2.running_mean',
          'encoder.4.0.bn2.running_var', 'encoder.4.0.shortcut.0.weight', 
          'encoder.4.0.shortcut.1.weight', 'encoder.4.0.shortcut.1.bias', 
          'encoder.4.0.shortcut.1.running_mean', 'encoder.4.0.shortcut.1.running_var', 
          'encoder.4.1.conv1.weight', 'encoder.4.1.bn1.weight', 'encoder.4.1.bn1.bias', 
          'encoder.4.1.bn1.running_mean', 'encoder.4.1.bn1.running_var', 'encoder.4.1.conv2.weight', 
          'encoder.4.1.bn2.weight', 'encoder.4.1.bn2.bias', 'encoder.4.1.bn2.running_mean',
          'encoder.4.1.bn2.running_var']

### Utility functions

In [9]:
def removekey(d, block):
    r = dict(d)
    copy = r.copy()
    toRemove = [base_to_remove]
    if block != 0:
        toRemove.append('encoder.5')
    if block == 2:
        toRemove.append('encoder.4')
    
    for key in copy.keys():
        if (key in toRemove[0]):
            print('key: {} is removed'.format(key))
            r.pop(key)
        if len(toRemove) > 1:
            if key.startswith(toRemove[1]):
                print('key: {} is removed'.format(key))
                r.pop(key)
        if len(toRemove) == 3:
            if key.startswith(toRemove[2]):
                print('key: {} is removed'.format(key))
                r.pop(key)
        
    return OrderedDict(r)

In [10]:
def load_model(modelToLoad, numberOfBlockToRemove):
    layers = base_layers - numberOfBlockToRemove
    hidden_sizes = [hidden_size] * layers
    num_blocks = [block_size] * layers
    model = Extractor(hidden_sizes, num_blocks)
    
    if cuda: model.cuda()
    
    checkpoint = torch.load(models_dir + '/' + modelToLoad, map_location=lambda storage, loc: storage)
    mod_weights = removekey(checkpoint, numberOfBlockToRemove)
    model.load_state_dict(mod_weights, strict=True)
    
    return model, mod_weights, checkpoint

### First experiment
Each pre-trained model is transformed in features extractor removing the final linear layer. 

#### "Pretrained model"

In [11]:
pretrainedNoLinear, _, _ = load_model(models[0], 0)

key: linear.weight is removed
key: linear.bias is removed


In [12]:
print(pretrainedNoLinear)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Finetuned model"

In [13]:
finetunedNoLinear, test, _ = load_model(models[1], 0)

key: linear.weight is removed
key: linear.bias is removed


In [14]:
print(finetunedNoLinear)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Clinical model"

In [15]:
clinicalNoLinear, _, _ = load_model(models[2], 0)

key: linear.weight is removed
key: linear.bias is removed


In [16]:
print(clinicalNoLinear)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

### Second experiment
Each pre-trained model is now transformed removing the last block of the original model.

#### "Pretrained model"

In [17]:
pretrainedNoLastBlock, _, _ = load_model(models[0], 1)

key: encoder.5.0.conv1.weight is removed
key: encoder.5.0.bn1.weight is removed
key: encoder.5.0.bn1.bias is removed
key: encoder.5.0.bn1.running_mean is removed
key: encoder.5.0.bn1.running_var is removed
key: encoder.5.0.conv2.weight is removed
key: encoder.5.0.bn2.weight is removed
key: encoder.5.0.bn2.bias is removed
key: encoder.5.0.bn2.running_mean is removed
key: encoder.5.0.bn2.running_var is removed
key: encoder.5.0.shortcut.0.weight is removed
key: encoder.5.0.shortcut.1.weight is removed
key: encoder.5.0.shortcut.1.bias is removed
key: encoder.5.0.shortcut.1.running_mean is removed
key: encoder.5.0.shortcut.1.running_var is removed
key: encoder.5.1.conv1.weight is removed
key: encoder.5.1.bn1.weight is removed
key: encoder.5.1.bn1.bias is removed
key: encoder.5.1.bn1.running_mean is removed
key: encoder.5.1.bn1.running_var is removed
key: encoder.5.1.conv2.weight is removed
key: encoder.5.1.bn2.weight is removed
key: encoder.5.1.bn2.bias is removed
key: encoder.5.1.bn2.runni

In [18]:
print(pretrainedNoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Finetuned model"

In [19]:
finetunedNoLastBlock, _, _ = load_model(models[1], 1)

key: encoder.5.0.conv1.weight is removed
key: encoder.5.0.bn1.weight is removed
key: encoder.5.0.bn1.bias is removed
key: encoder.5.0.bn1.running_mean is removed
key: encoder.5.0.bn1.running_var is removed
key: encoder.5.0.bn1.num_batches_tracked is removed
key: encoder.5.0.conv2.weight is removed
key: encoder.5.0.bn2.weight is removed
key: encoder.5.0.bn2.bias is removed
key: encoder.5.0.bn2.running_mean is removed
key: encoder.5.0.bn2.running_var is removed
key: encoder.5.0.bn2.num_batches_tracked is removed
key: encoder.5.0.shortcut.0.weight is removed
key: encoder.5.0.shortcut.1.weight is removed
key: encoder.5.0.shortcut.1.bias is removed
key: encoder.5.0.shortcut.1.running_mean is removed
key: encoder.5.0.shortcut.1.running_var is removed
key: encoder.5.0.shortcut.1.num_batches_tracked is removed
key: encoder.5.1.conv1.weight is removed
key: encoder.5.1.bn1.weight is removed
key: encoder.5.1.bn1.bias is removed
key: encoder.5.1.bn1.running_mean is removed
key: encoder.5.1.bn1.run

In [20]:
print(finetunedNoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Clinical model"

In [21]:
clinicalNoLastBlock, _, _ = load_model(models[2], 1)

key: encoder.5.0.conv1.weight is removed
key: encoder.5.0.bn1.weight is removed
key: encoder.5.0.bn1.bias is removed
key: encoder.5.0.bn1.running_mean is removed
key: encoder.5.0.bn1.running_var is removed
key: encoder.5.0.conv2.weight is removed
key: encoder.5.0.bn2.weight is removed
key: encoder.5.0.bn2.bias is removed
key: encoder.5.0.bn2.running_mean is removed
key: encoder.5.0.bn2.running_var is removed
key: encoder.5.0.shortcut.0.weight is removed
key: encoder.5.0.shortcut.1.weight is removed
key: encoder.5.0.shortcut.1.bias is removed
key: encoder.5.0.shortcut.1.running_mean is removed
key: encoder.5.0.shortcut.1.running_var is removed
key: encoder.5.1.conv1.weight is removed
key: encoder.5.1.bn1.weight is removed
key: encoder.5.1.bn1.bias is removed
key: encoder.5.1.bn1.running_mean is removed
key: encoder.5.1.bn1.running_var is removed
key: encoder.5.1.conv2.weight is removed
key: encoder.5.1.bn2.weight is removed
key: encoder.5.1.bn2.bias is removed
key: encoder.5.1.bn2.runni

In [22]:
print(clinicalNoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

### Third experiment
Each pre-trained model is now transformed removing the last two blocks.

#### "Pretrained model"

In [23]:
pretrainedNoTwoLastBlock, _, _ = load_model(models[0], 2)

key: encoder.4.0.conv1.weight is removed
key: encoder.4.0.bn1.weight is removed
key: encoder.4.0.bn1.bias is removed
key: encoder.4.0.bn1.running_mean is removed
key: encoder.4.0.bn1.running_var is removed
key: encoder.4.0.conv2.weight is removed
key: encoder.4.0.bn2.weight is removed
key: encoder.4.0.bn2.bias is removed
key: encoder.4.0.bn2.running_mean is removed
key: encoder.4.0.bn2.running_var is removed
key: encoder.4.0.shortcut.0.weight is removed
key: encoder.4.0.shortcut.1.weight is removed
key: encoder.4.0.shortcut.1.bias is removed
key: encoder.4.0.shortcut.1.running_mean is removed
key: encoder.4.0.shortcut.1.running_var is removed
key: encoder.4.1.conv1.weight is removed
key: encoder.4.1.bn1.weight is removed
key: encoder.4.1.bn1.bias is removed
key: encoder.4.1.bn1.running_mean is removed
key: encoder.4.1.bn1.running_var is removed
key: encoder.4.1.conv2.weight is removed
key: encoder.4.1.bn2.weight is removed
key: encoder.4.1.bn2.bias is removed
key: encoder.4.1.bn2.runni

In [24]:
print(pretrainedNoTwoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Finetuned model"

In [25]:
finetunedNoTwoLastBlock, _, _ = load_model(models[1], 2)

key: encoder.4.0.conv1.weight is removed
key: encoder.4.0.bn1.weight is removed
key: encoder.4.0.bn1.bias is removed
key: encoder.4.0.bn1.running_mean is removed
key: encoder.4.0.bn1.running_var is removed
key: encoder.4.0.bn1.num_batches_tracked is removed
key: encoder.4.0.conv2.weight is removed
key: encoder.4.0.bn2.weight is removed
key: encoder.4.0.bn2.bias is removed
key: encoder.4.0.bn2.running_mean is removed
key: encoder.4.0.bn2.running_var is removed
key: encoder.4.0.bn2.num_batches_tracked is removed
key: encoder.4.0.shortcut.0.weight is removed
key: encoder.4.0.shortcut.1.weight is removed
key: encoder.4.0.shortcut.1.bias is removed
key: encoder.4.0.shortcut.1.running_mean is removed
key: encoder.4.0.shortcut.1.running_var is removed
key: encoder.4.0.shortcut.1.num_batches_tracked is removed
key: encoder.4.1.conv1.weight is removed
key: encoder.4.1.bn1.weight is removed
key: encoder.4.1.bn1.bias is removed
key: encoder.4.1.bn1.running_mean is removed
key: encoder.4.1.bn1.run

In [26]:
print(finetunedNoTwoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=

#### "Clinical model"

In [27]:
clinicalNoTwoLastBlock, _, _ = load_model(models[2], 2)

key: encoder.4.0.conv1.weight is removed
key: encoder.4.0.bn1.weight is removed
key: encoder.4.0.bn1.bias is removed
key: encoder.4.0.bn1.running_mean is removed
key: encoder.4.0.bn1.running_var is removed
key: encoder.4.0.conv2.weight is removed
key: encoder.4.0.bn2.weight is removed
key: encoder.4.0.bn2.bias is removed
key: encoder.4.0.bn2.running_mean is removed
key: encoder.4.0.bn2.running_var is removed
key: encoder.4.0.shortcut.0.weight is removed
key: encoder.4.0.shortcut.1.weight is removed
key: encoder.4.0.shortcut.1.bias is removed
key: encoder.4.0.shortcut.1.running_mean is removed
key: encoder.4.0.shortcut.1.running_var is removed
key: encoder.4.1.conv1.weight is removed
key: encoder.4.1.bn1.weight is removed
key: encoder.4.1.bn1.bias is removed
key: encoder.4.1.bn1.running_mean is removed
key: encoder.4.1.bn1.running_var is removed
key: encoder.4.1.conv2.weight is removed
key: encoder.4.1.bn2.weight is removed
key: encoder.4.1.bn2.bias is removed
key: encoder.4.1.bn2.runni

In [28]:
print(clinicalNoTwoLastBlock)

Extractor(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): ResidualBlock(
        (conv1): Conv1d(64, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv1d(64, 100, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): ResidualBlock(
        (conv1): Conv1d(100, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn1): BatchNorm1d(100, eps=